In [1]:
import json
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
# from keras.applications.resnet50 import ResNet50
# from keras.applications import xception
# from keras.applications import inception_v3

%matplotlib inline

/Users/matt/dev/ML/kaggle-fashion/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open("../data/train.json") as train:
    train_json = json.load(train)

df1 = pd.DataFrame(train_json['images'])
df2 = pd.DataFrame(train_json['annotations'])

# df1.head()

# random_seed = 101

# # Mac/Linux file paths
# # TRAIN = glob("./data/train/*jpg")
# # TEST = glob("./data/test/*jpg")
# # df = pd.read_csv("./data/train.csv")
# # df["Image"] = df["Image"].map( lambda x : "./data/train/"+x)

# ROOT = "../data/"
# TRAIN = glob("../data/train/*jpg")
# TEST = glob("../data/test/*jpg")
# VALIDATION = glob("../data/validation/*jpg")

# with open("../data/train.json") as datafile1:
#     train_data = json.load(datafile1)
    
# with open("../data/test.json") as datafile2:
#     test_data = json.load(datafile2)
    
# with open("../data/validation.json") as datafile3:
#     valid_data = json.load(datafile3)
    
    
# # df = 

# # ImageToLabelDict = dict( zip( df["Image"], df["Id"]))

# # def ImportImage(filename):
# #     """
# #     Import images, converting them to black and white with consistent size
# #     """
# #     img = Image.open(filename).convert("LA").resize((SIZE,SIZE))
# #     return np.array(img)[:,:,0]

# # x = np.array([ImportImage(img) for img in TRAIN])

In [4]:
train=pd.merge(df1, df2, on='imageId', how='inner')

In [6]:
train['labelId'].dtype

dtype('O')